# YOLO + Mediapipe VOLUME CONTROL WITH YOLOV5 MODEL TRAIN USING OUR DATASET

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import os
from ultralytics import YOLO

def set_volume_macos(volume_percent):
    volume_percent = max(0, min(100, int(volume_percent))) 
    os.system(f"osascript -e 'set volume output volume {volume_percent}'")

model = YOLO('./best.pt')

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from camera. Exiting...")
        break

    results = model.predict(frame, imgsz=640)  

    results = results[0]  

    detections = results.boxes

    if detections is not None:
        for box in detections:
            bbox = box.xyxy[0]
            x1, y1, x2, y2 = map(int, bbox.tolist())

            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            if cls == 0:  
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                roi = frame[y1:y2, x1:x2]

                rgb_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                result = hands.process(rgb_roi)

                if result.multi_hand_landmarks:
                    for hand_landmarks in result.multi_hand_landmarks:
                        mp.solutions.drawing_utils.draw_landmarks(
                            roi, hand_landmarks, mp_hands.HAND_CONNECTIONS
                        )
                        thumb_tip = hand_landmarks.landmark[4]
                        index_tip = hand_landmarks.landmark[8]
                        palm_base = hand_landmarks.landmark[0]
                        middle_tip = hand_landmarks.landmark[12]

                        h_roi, w_roi, _ = roi.shape
                        thumb_pos = np.array([int(thumb_tip.x * w_roi), int(thumb_tip.y * h_roi)])
                        index_pos = np.array([int(index_tip.x * w_roi), int(index_tip.y * h_roi)])
                        palm_pos = np.array([int(palm_base.x * w_roi), int(palm_base.y * h_roi)])
                        middle_pos = np.array([int(middle_tip.x * w_roi), int(middle_tip.y * h_roi)])

                        cv2.circle(roi, tuple(thumb_pos), 10, (255, 0, 0), -1)
                        cv2.circle(roi, tuple(index_pos), 10, (255, 0, 0), -1)
                        cv2.circle(roi, tuple(palm_pos), 10, (0, 255, 0), -1)
                        cv2.circle(roi, tuple(middle_pos), 10, (0, 255, 0), -1)

                        thumb_index_gap = np.linalg.norm(thumb_pos - index_pos)
                        hand_length = np.linalg.norm(palm_pos - middle_pos)
                        normalized_gap = thumb_index_gap / hand_length if hand_length != 0 else 0
                        vol_perc = np.interp(normalized_gap, [0.1, 0.5], [0, 100])

                        set_volume_macos(vol_perc)
                        cv2.putText(
                            frame,
                            f'Volume: {int(vol_perc)}%',
                            (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (255, 0, 0),
                            2,
                        )

    cv2.imshow("Hand Detection and Volume Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
